In [1]:
import re
import sys; sys.path.append("../")

import numpy as np
import pandas as pd

from lib.columns import rearrange_complaint_columns
from lib.textract import parse_textract_datetime

In [2]:
cprr = pd.read_csv("../data/Baton Rouge Police Department/Baton Rouge_PD_CPRR_2018.csv", encoding="latin1")
cprr.rename(columns=lambda x: x.strip(), inplace=True)
cprr.rename(columns={"Comptaint": "Complaint"}, inplace=True)

# manual edits
cprr.iloc[4, 2] = "Administrative Review"
cprr.iloc[12, 5] = cprr.iloc[12, 5] + "P10467 Patrol 3rd District"
cprr.iloc[12, 6] = cprr.iloc[12, 6] + "Orders - (Pursuit) - 40"
for ind in [16, 23, 25, 29]:
    m = re.match(r"^(.+) (\d+\:\d+) *$", cprr.iloc[ind-1, 6])
    cprr.iloc[ind-1, 6] = m.group(1)
    cprr.iloc[ind, 6] = "%s %s" % (m.group(2), cprr.iloc[ind, 6])
for ind in [39]:
    m = re.match(r"^(\d+) (.+)$", cprr.iloc[ind+1, 6])
    cprr.iloc[ind, 6] = cprr.iloc[ind, 6] + m.group(1)
    cprr.iloc[ind+1, 6] = m.group(2)
cprr.iloc[24, 5] = cprr.iloc[24, 5][21:]
cprr.iloc[38, 5] = cprr.iloc[38, 5] + "Operational Services"
cprr.iloc[66, 6] = cprr.iloc[66, 6] + "Orders - (Pursuit) - 40"
cprr.iloc[72, 0] = "2018"
cprr.iloc[72, 1] = "35"
cprr.iloc[72, 2] = "Administrative Review"
cprr.iloc[72, 5] = cprr.iloc[72, 5][:-1]
cprr.iloc[72, 6] = cprr.iloc[72, 6][:-1]
cprr.iloc[83, 5] = "%s%s" % (cprr.iloc[83, 5], cprr.iloc[84, 5])
cprr.iloc[83, 6] = "%s%s" % (cprr.iloc[83, 6], cprr.iloc[84, 6])
cprr.iloc[128, 5] = "%s%s" % (cprr.iloc[128, 5], cprr.iloc[133, 4])
cprr.iloc[128, 6] = "%s%s" % (cprr.iloc[128, 6], cprr.iloc[133, 5])
cprr.iloc[115, 6] = "%s%s" % (cprr.iloc[115, 6], cprr.iloc[129, 6])
for ind in [134, 135]:
    cprr.iloc[ind, 6] = cprr.iloc[ind, 5]
    cprr.iloc[ind, 5] = cprr.iloc[ind, 4]
    cprr.iloc[ind, 4] = ""
cprr.iloc[142, 5] = cprr.iloc[144, 5]
cprr.iloc[143, 5] = cprr.iloc[144, 5]

# insert missing rows in page 2
cprr.loc[len(cprr)] = [
    "2018", "010", "Administrative Review", "1/26/18", "1/26/18",
    "ROBERTSON JASON R., P10639 Patrol 1st District",
    "3:17 Carrying Out Orders / General Orders - (Pursuit) - 40",
    "Not Sustained", "Not Sustained"]
cprr.loc[len(cprr)] = [
    "2018", "011", "Administrative Review", "2/8/18", "2/8/18",
    "CARTER. JR. DARRELL J., P10511 Patrol 4th District",
    "3:20 Use of Force / Intermediate Weapon - 52",
    "Not Sustained", "Not Sustained"]

# normalize
cprr.loc[:, "IA Year"] = cprr["IA Year"].str.replace("-", "").str.strip()
cprr.loc[:, "Status"] = cprr["Status"].str.replace("-", "").str.strip()\
    .str.title()
cprr.loc[:, "Received"] = cprr["Received"].str.replace("-", "").str.strip()
cprr.loc[:, "Occur Date"] = cprr["Occur Date"].str.replace("-", "")\
    .str.strip().fillna("")
cprr.loc[:, "Officer Name"] = cprr["Officer Name"].str.strip()\
    .str.replace(r"(\.|,) +(PC?\d+)", r", \2")\
    .str.replace(", JR", ". JR", regex=False)\
    .str.replace(" Il ", " II ", regex=False)\
    .str.replace(r"(\d+)\. ", r"\1 ")\
    .str.replace("Distnct", "District", regex=False)\
    .str.replace("none entered", "", regex=False).fillna("")
cprr.loc[:, "Complaint"] = cprr.loc[:, "Complaint"]\
    .str.replace(r"\< *\d+.+$", "").str.strip()\
    .str.replace(r"^(\d+)(?:\.|,)(\d+)", r"\1:\2")\
    .str.replace(r" \.(\d+)", r" \1")\
    .str.replace(r" of$", "").str.replace("Use Force", "Use of Force", regex=False)\
    .str.replace(r" ?- ", " ").str.replace(" lo ", " to ", regex=False)\
    .str.replace("Emply", "Empty", regex=False)\
    .str.replace("Oul ", "Out ", regex=False).fillna("")
cprr.loc[:, "Action"] = cprr["Action"]\
    .where(~cprr["Action"].str.isupper().fillna(False), cprr["Action"].str.title())\
    .str.replace(" lo ", " to ", regex=False)\
    .str.replace(r"^ *- *$", "")\
    .str.replace("Suspension.", "Suspension,", regex=False)\
    .str.replace("Exonerated;", "Exonerated:", regex=False)\
    .str.replace("Sgl. ", "Sgt. ", regex=False).str.strip().fillna("")
cprr.loc[:, "Disposition"] = cprr["Disposition"].str.replace(r" ?- ", " ")\
    .str.replace(r"\< *\d+.+$", "").str.replace(r"^\.", "")\
    .str.replace("Exoneraled", "Exonerated", regex=False)\
    .str.replace("Sustaned", "Sustained", regex=False).str.strip().fillna("")

cprr.drop([84, 129, 133], inplace=True)
cprr.loc[:, "1A Seq"] = cprr["1A Seq"].str.rjust(3, "0")
cprr.reset_index(drop=True, inplace=True)
cprr


,IA Year,1A Seq,Status,Received,Occur Date,Officer Name,Complaint,Action,Disposition
0,2018,002,Administrative Review,1/2/18,12/30/17,"BARCELONA JESSE R, P1254 Cib Street Crimes Unit",3:17 Carrying Out Orders / General Orders (Pur...,Exonerated,Exonerated
1,2018,004,Administrative Review,1/9/18,1/9/18,"ACREE JASON K, P10280 Cib Narcotics",3:17 Carrying Out Orders / General Orders (Pur...,Not Sustained,Not Sustained
2,2018,003,Office Investigation,1/9/18,12/3/17,"RIMES MELISSA J, P10476 Op Service Comm Center",2:4 Abuse of Sick Leave 66,Resigned,Office Investigation
3,2018,003,Office Investigation,1/9/18,12/3/17,"RIMES MELISSA J, P10476 Op Service Comm Center",2:4 Abuse of Sick Leave 66,Office Investigation,Office Investigation
4,2018,005,Administrative Review,1/10/18,1/10/18,"HAMADEH YUSEFF G, P10578 Cib Cib",3:20 Use of Force / Hard Empty Hand 53,Not Sustained,Not Sustained
...,...,...,...,...,...,...,...,...,...
144,2019,003,Administrative Review,1/30/19,3/27/18,"CLARK KENNETH M, P1245 Patrol 2nd District",3:17 Carrying Out Orders / General Orders (Pur...,Letter of Reprimand,Sustained
145,2019,003,Administrative Review,1/30/19,3/27/18,"CLARK KENNETH M, P1245 Patrol 2nd District",3:18 Insubordination 43,Letter of Reprimand,Not Sustained
146,2019,003,Administrative Review,1/30/19,3/27/18,"CLARK KENNETH M, P1245 Patrol 2nd District",2:12 Conduct Unbecoming an Officer 21,Letter of Reprimand,Sustained
147,2018,010,Administrative Review,1/26/18,1/26/18,"ROBERTSON JASON R., P10639 Patrol 1st District",3:17 Carrying Out Orders / General Orders (Pur...,Not Sustained,Not Sustained


In [3]:
dep = cprr["Officer Name"].str.replace(r"^(.+), (PC?\d+) (.+)$", r"\1 # \2 # \3").str.split(" # ", expand=True)
dep.columns = ["Name", "Department #", "Department"]
dep.loc[:, "Name"] = dep["Name"].str.replace(r"\.", "").str.strip().str.title()

names = dep["Name"].str.replace(r"^(\w+(?: (?:Iii?|Iv|V|Jr|Sr))?) (\w+)(?: (\w+|N\/A))?$", r"\1 # \2 # \3").str.split(" # ", expand=True)
names.columns = ["Last Name", "First Name", "Middle Name"]
names.loc[:, "Middle Name"] = names["Middle Name"].str.replace("N/A", "", regex=False).fillna("")
names.loc[:, "Last Name"] = names["Last Name"]\
    .str.replace(r"^(.+) (Iii?|Iv)$", lambda m: "%s %s" % (m.group(1), m.group(2).upper())).fillna("")
names.loc[:, "First Name"] = names["First Name"].fillna("")

df = pd.concat([dep, names], axis=1)
df

,Name,Department #,Department,Last Name,First Name,Middle Name
0,Barcelona Jesse R,P1254,Cib Street Crimes Unit,Barcelona,Jesse,R
1,Acree Jason K,P10280,Cib Narcotics,Acree,Jason,K
2,Rimes Melissa J,P10476,Op Service Comm Center,Rimes,Melissa,J
3,Rimes Melissa J,P10476,Op Service Comm Center,Rimes,Melissa,J
4,Hamadeh Yuseff G,P10578,Cib Cib,Hamadeh,Yuseff,G
...,...,...,...,...,...,...
144,Clark Kenneth M,P1245,Patrol 2nd District,Clark,Kenneth,M
145,Clark Kenneth M,P1245,Patrol 2nd District,Clark,Kenneth,M
146,Clark Kenneth M,P1245,Patrol 2nd District,Clark,Kenneth,M
147,Robertson Jason R,P10639,Patrol 1st District,Robertson,Jason,R


In [4]:
cprr["Complaint"].unique()
# "rule_violation_code rule_violation_text / paragraph_violation paragraph_number?"

array(['3:17 Carrying Out Orders / General Orders (Pursuit) 40',
       '2:4 Abuse of Sick Leave 66',
       '3:20 Use of Force / Hard Empty Hand 53',
       '3:12 Confiscated Property 35',
       '2:12 Conduct Unbecoming an Officer 21', '3:23 Truthfulness 58',
       '2:1 Use of Alcohol/Drugs (Drugs) 12',
       '3:22 Violation of Known Laws / Violations 56',
       '2:2 Command of Temper 13',
       '3:20 Use of Force / Soft Empty Hand 54',
       '3:20 Use of Force / Shooting (Non Contact) 47',
       '2:14 DMVR Violation 68',
       '3:4 Failure to Secure Property or Evidence 26',
       '1:7 Comp/Submission of Forms 8', '2:9 Traffic Violations 19',
       '3:17 Carrying Out Orders General Orders (Pursuit) 40',
       '3:17 Carrying Out Orders / Memorandums 41',
       '3:20 Use of Force / Shooting (Animal) 48',
       '2:8 Damaging Department Equipment 18',
       '2:11 Outside Agency Involvement 67', '2:3 Shirking Duties 14',
       '2:13 Respect of Fellow Officers 22',
       '3

In [5]:
cprr["Action"].unique()

array(['Exonerated', 'Not Sustained', 'Resigned', 'Office Investigation',
       'Hold in abeyance', 'Counseled', '', 'Not Sustained (60 day rule)',
       '30-day suspension', 'Letter of Caution',
       'Command of Temper (2-Day Suspension, Verbal Judo Class, Attaining Respect class, Early Intervention',
       'Tactical Training in Reasonable Suspicion & Felony Stops',
       'Tactical Training in Reasonable Suspicion & Felony Slops',
       'Letter of Reprimand', 'Peer Intervention Training',
       '21-Day Suspension', 'Peer intervention Training',
       'Verbal Judo Training', '1-Day Suspension', 'Termination',
       'Letter of Caution; Firearm Safety Training; Range Master',
       'Exonerated: Not Sustained',
       'Letter of Caution & 1 day driving school',
       'Letter of Instruction', 'Conference Worksheet',
       'Conference worksheet', '9-Day Suspension', '2-Day Suspension',
       '65-day suspension', 'Demotion - From Sgt. to Cpl.',
       '7 day suspension; De- Esc

In [6]:
cprr["Disposition"].unique()

array(['Exonerated', 'Not Sustained', 'Office Investigation', 'Sustained',
       'Admin Review', ''], dtype=object)

In [7]:
cprr["Status"].unique()

array(['Administrative Review', 'Office Investigation', 'Referral'],
      dtype=object)